# ReAção Server

This notebook implements the ReAção emotion detection server. The server works with two threads, the first responsible for interacting with the final application, and the second for reading the camera and detecting emotion.

In [61]:
import socket
import json
import select
import threading
from collections import deque

import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
import torch
from torchvision import transforms 
from torch.autograd import Variable

import sys
emonet_path = "D:\\Github\\emonet" # Change for your emonet repository path
sys.path.append(emonet_path)
from emonet.models import EmoNet

Global variables

In [3]:
stop_sever = False #If should stop the server thread
stop_detector = False #If should stop the detector thread

emotion_data = {"valence": 0.0, "arousal":0.0, "meanValence":0.0, "meanArousal":0.0} #Emotion info
tracking = False #If should track the emotions

## Settings

Change the address, port and camera index as needed:

In [2]:
HOST = "127.0.0.1"
PORT = 65444
capture_device_index = 1

## Server function

The server function creates the connection with the client, get tracking info and send emotion data in JSON format:

In [57]:
def server_func():
    global tracking, stop_sever, emotion_data
    tracking = False
    
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((HOST, PORT))

    s.listen()
    conn = None

    #Create connection
    while conn is None:
        if stop_sever:
            s.close()
            return

        readable, _, _ = select.select([s], [], [], 0)

        for rconn in readable:
            if rconn is s:
                conn, addr = s.accept()
                break

    #Get and send data
    while True:
        if stop_sever:
            conn.close()
            s.close()
            return

        rconn, _, _ = select.select([conn], [], [], 0)

        if tracking:
            data = json.dumps(emotion_data).encode("UTF-8")
            conn.sendall(data)

        if len(rconn) != 0:
            message = rconn[0].recv(1024)
            message = message.decode("utf-8")
            
            if message == "false":
                tracking = False
                data = json.dumps(emotion_data).encode("UTF-8")
                conn.sendall(data)
            else:
                tracking = True

            print("Tracking:", tracking)
    

server_thread = threading.Thread(target=server_func, daemon=True)
server_thread.start()

Tracking: True


Exception in thread Thread-16:
Traceback (most recent call last):
  File "c:\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-57-bcc9b33f07d3>", line 35, in server_func
ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto


## Detector

For the detector side, first the model is loaded:

In [5]:
#Model constants

torch.backends.cudnn.benchmark =  True
n_expression = 8
device = 'cuda:0'
image_size = 256
expression_dict = {0: 'neutral', 1:'happy', 2:'sad', 3:'surprise', 4:'fear', 5:'disgust', 6:'anger', 7:'contempt', 8:'none'}
state_dict_path = emonet_path+"\\pretrained\\emonet_8.pth"

In [6]:
#Model preprocessing

test_transforms = transforms.Compose([
                                        transforms.Resize((256,256)),
                                        transforms.ToTensor(),
                                     ])

In [7]:
# Load model

state_dict = torch.load(str(state_dict_path), map_location='cpu')
state_dict = {k.replace('module.',''):v for k,v in state_dict.items()}
net = EmoNet(n_expression=n_expression).to(device)
net.load_state_dict(state_dict, strict=False)
net.eval()

And then the detector function gets the camera frame and saves the detected emotion in the global dict:

In [62]:
def detector_func():
    global tracking, stop_detector, emotion_data
    #tracking = False
    
    cap = cv.VideoCapture(capture_device_index)
    ret, frane = cap.read()

    valence_hist = deque(maxlen=60)
    arousal_hist = deque(maxlen=60)


    while cap.isOpened():
        if stop_detector:
            cap.release()
            return
        if not tracking:
            if len(valence_hist) != 0:
                valence_hist.clear()
                arousal_hist.clear()

            continue

        ret, frame = cap.read()

        if not ret:
            print("Camera stream end")
            break


        frameRGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image = Image.fromarray(frameRGB)

        image_tensor = test_transforms(image).float()
        image_tensor = image_tensor.unsqueeze(0)

        imgInput = Variable(image_tensor)
        imgInput = imgInput.to(device)

        output = net(imgInput)

        valence = output["valence"]
        valence = valence.cpu().detach().numpy()
        valence = valence[0]

        arousal = output["arousal"]
        arousal = arousal.cpu().detach().numpy()
        arousal = arousal[0]

        #heatmap = output['heatmap']
        #heatmap = heatmap.cpu().detach().numpy()

        #expression = output['expression']
        #expression = expression.cpu().detach().numpy()
        #expression = np.argmax(np.squeeze(expression), axis=0)
        #expression = expression_dict[expression]

        valence_hist.append(valence)
        arousal_hist.append(arousal)

        emotion_data["valence"] = float(valence)
        emotion_data["arousal"] = float(arousal)
        emotion_data["meanArousal"] = float(np.mean(valence_hist))
        emotion_data["meanValence"] = float(np.mean(arousal_hist))




detector_thread = threading.Thread(target=detector_func, daemon=True)
detector_thread.start()

## Monitoring

The following cell can be useful to monitor if the threads are still working: 

In [63]:
while True:
    stop = False

    if not detector_thread.is_alive():
        print("Detector thread is dead")
        stop = True
        
    if not server_thread.is_alive():
        print("Server thread is dead")
        stop = True
    
    if stop:
        break
        

Server thread is dead


You can also use the stop variables to stop some thread:

In [59]:
stop_detector = False
stop_sever = False

In [38]:
tracking=True

In [47]:
stop_sever = True